In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_excel('result.xlsx')[0:32]
data

In [ ]:
X=data.iloc[:,1:4]

In [ ]:
X=np.array(X)

In [ ]:
len(X)

In [ ]:
y=data[['phase/ 0 alpha/1 beta/2 mix']]

In [ ]:
y=np.array(y).ravel()

In [ ]:
y

In [ ]:
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import entropy
import torchbnn as bnn


In [ ]:

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)


# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_accuracies = []

for train_index, val_index in kf.split(X):
    # Split data
    X_train, X_val = X_tensor[train_index], X_tensor[val_index]
    y_train, y_val = y_tensor[train_index], y_tensor[val_index]
    
    # Define BNN model
    model = nn.Sequential(
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=X.shape[1], out_features=50),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=50, out_features=len(np.unique(y)))
    )
    
    criterion = nn.CrossEntropyLoss()
    kl_loss = bnn.BKLLoss(reduction='mean')
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    # Train model
    for epoch in range(500):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train) + 0.1 * kl_loss(model)
        loss.backward()
        optimizer.step()
    
    # Evaluate model
    model.eval()
    with torch.no_grad():
        val_preds = model(X_val).argmax(dim=1)
        acc = (val_preds == y_val).float().mean().item()
        cv_accuracies.append(acc)


print("Mean Accuracy:", np.mean(cv_accuracies))
std_cv_acc=np.std(cv_accuracies)

sem_cv_acc =std_cv_acc  / np.sqrt(len(cv_accuracies))
print("Accuracy SEM:", sem_cv_acc)
